In [ ]:
import os
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import utils as vutils
from discriminator import Discriminator
from lpips import LPIPS
from vqgan import VQGAN
from utils import load_data, weights_init
import lightning as L
from vqgan_lightning import LitVQGAN
import json
from utils import ImagePaths
from torch.utils.data import Dataset, DataLoader

from matplotlib.pyplot import plt

from ..

/home/oyoungquist/.conda/envs/taming/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # pull out the current working directory
# cwd = os.getcwd()

# # appened the directory with actual images
# new_path_prefix = os.path.join(cwd, "..", "processed_data", "reprocessed_resizedimages_revised.json")
# print(new_path_prefix)

# # open the json and fix it first
# json_df = pd.read_json(new_path_prefix)

# # extract negative file-paths
# negative_image_paths = json_df[json_df["time_step_label"] == 1].image_file_path.to_list()

# new_file_path = os.path.join(cwd, "..", "processed_data", "negative_image_paths.txt")
# # write out negative data
# with open(new_file_path, "w+") as f:
#     for line in negative_image_paths:
#         f.write(f"{line}\n")

In [3]:
cwd = os.getcwd()
model_folder_path = os.path.join(cwd, "training_results", "03_22_202416_44_55")
model_file_path = os.path.join(model_folder_path, "lightning_logs", "version_0", "checkpoints", "epoch=12-step=1083342.ckpt")
negative_data_path = os.path.join(cwd, "..", "processed_data", "negative_image_paths.txt")

eval_output_path = os.path.join(model_folder_path, "negative_images")

if not os.path.exists(eval_output_path):
    os.makedirs(eval_output_path)

In [4]:
# load negative image data
test_data = ImagePaths(negative_data_path, size=256)
negative_data_loader = DataLoader(test_data, batch_size=1, shuffle=False, num_workers=1)

Loading data from location: /home/oyoungquist/Research/CPFP/VQGAN-pytorch-hcr/../processed_data/negative_image_paths.txt
Loaded 37440 images


In [5]:
# load trained model
vqgan = LitVQGAN.load_from_checkpoint(model_file_path)

/home/oyoungquist/.conda/envs/taming/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/oyoungquist/.conda/envs/taming/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth


In [6]:
vqgan.eval()

LitVQGAN(
  (vqgan): VQGAN(
    (encoder): Encoder(
      (model): Sequential(
        (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ResidualBlock(
          (block): Sequential(
            (0): GroupNorm(
              (gn): GroupNorm(32, 128, eps=1e-06, affine=True)
            )
            (1): Swish()
            (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (3): GroupNorm(
              (gn): GroupNorm(32, 128, eps=1e-06, affine=True)
            )
            (4): Swish()
            (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
        (2): ResidualBlock(
          (block): Sequential(
            (0): GroupNorm(
              (gn): GroupNorm(32, 128, eps=1e-06, affine=True)
            )
            (1): Swish()
            (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (3): GroupNorm(
              (gn): GroupNorm

In [ ]:
for im in negative_data_loader:
    recons = vqgan.encode_GAT_nodes(im)
    diff = torch.abs(recons - im)

    im_np = im.tolist()
    recons_np = recons.tolist()
    diff_np = diff.tolist()

    fig, axes = plt.subplots(nrows=1, ncols=3, squeeze=True)
    
    axes[0].imshow(log_dict["inputs"][0].numpy())
    axes[0].axis('off')
    axes[0].set_title("Input")
    axes[1].imshow(log_dict["reconstructions"][0].numpy())
    axes[1].axis('off')
    axes[1].set_title("Recon.")
    axes[2].imshow(log_dict["feature_diff"][0].numpy())
    axes[2].axis('off')
    axes[2].set_title("Diff.")
    new_path = os.path.join(output_path, "neg_eval_sample_{:d}.png".format(batch_num))
    batch_num += 1
    fig.savefig(new_path)
    plt.close()

